MTG Card Identifier  

In [2]:
# --- Imports & Constants ---
import requests
import pandas as pd
from IPython.display import display

SCRYFALL_API_URL = "https://api.scryfall.com/cards/named"
FRANKFURTER_API_URL = "https://api.frankfurter.app/latest"


In [3]:
def fetch_card_data(card_name: str) -> dict:
    """
    Fetches card data from Scryfall API.
    
    Args:
        card_name (str): Exact name of the MTG card.
        
    Returns:
        dict: Extracted card information.
    """
    params = {"exact": card_name}
    response = requests.get(SCRYFALL_API_URL, params=params)
    response.raise_for_status()
    
    data = response.json()
    return {
        "Name": data.get("name"),
        "Set": data.get("set_name"),
        "Rarity": data.get("rarity"),
        "Oracle Text": data.get("oracle_text"),
        "Image URL": data.get("image_uris", {}).get("normal"),
        "Price USD": data.get("prices", {}).get("usd")
    }


In [4]:
def get_usd_to_eur_rate() -> float:
    """
    Fetches the current USD to EUR exchange rate.
    
    Returns:
        float: USD to EUR rate.
    """
    params = {"from": "USD", "to": "EUR"}
    response = requests.get(FRANKFURTER_API_URL, params=params)
    response.raise_for_status()
    
    data = response.json()
    return data["rates"]["EUR"]


In [5]:
def convert_usd_to_eur(usd_price, rate) -> float | None:
    """
    Converts USD price to EUR using given rate.
    
    Args:
        usd_price (str or float): USD price.
        rate (float): USD to EUR exchange rate.
        
    Returns:
        float or None: Converted EUR price or None if invalid.
    """
    try:
        usd_value = float(usd_price)
        return round(usd_value * rate, 2)
    except (TypeError, ValueError):
        print("⚠️  Warnung: Ungültiger USD-Preis, EUR-Spalte bleibt leer.")
        return None


In [6]:
def save_dataframe_as_csv(df: pd.DataFrame, filename: str) -> None:
    """
    Saves given DataFrame as CSV.
    
    Args:
        df (pd.DataFrame): Data to save.
        filename (str): Destination filename.
    """
    df.to_csv(filename, index=False)
    print(f"✅ CSV gespeichert: {filename}")


In [7]:
def build_card_dataframe(card_info: dict, eur_price: float | None) -> pd.DataFrame:
    """
    Builds a pandas DataFrame from card info and EUR price.
    
    Args:
        card_info (dict): Card data.
        eur_price (float | None): Converted EUR price.
        
    Returns:
        pd.DataFrame: Formatted DataFrame.
    """
    df = pd.DataFrame([card_info])
    df["Price EUR"] = eur_price
    
    # Define column order
    columns_order = ["Name", "Set", "Rarity", "Price USD", "Price EUR", "Oracle Text", "Image URL"]
    df = df[[col for col in columns_order if col in df.columns]]
    
    return df


In [8]:
# --- Pipeline Execution ---

sample_card_name = "Lightning Bolt"

try:
    print(f"\n🔍 Suche Karte: '{sample_card_name}'...")
    
    # Fetch card data
    card_info = fetch_card_data(sample_card_name)
    print("✅ Kartendaten erfolgreich abgerufen.")
    
    # Get exchange rate
    rate = get_usd_to_eur_rate()
    print(f"💱 Aktueller USD→EUR-Kurs: {rate:.4f}")
    
    # Convert price
    eur_price = convert_usd_to_eur(card_info["Price USD"], rate)
    
    # Build DataFrame
    df = build_card_dataframe(card_info, eur_price)
    
    # Display
    display(df)
    
    # Save CSV
    filename = sample_card_name.replace(" ", "_") + "_info.csv"
    save_dataframe_as_csv(df, filename)
    
except requests.HTTPError as e:
    status_code = e.response.status_code if e.response else "N/A"
    print(f"❌ HTTP-Fehler beim Abrufen der Karte ({status_code}): {e}")
except Exception as e:
    print(f"❌ Unerwarteter Fehler: {e}")



🔍 Suche Karte: 'Lightning Bolt'...
✅ Kartendaten erfolgreich abgerufen.
💱 Aktueller USD→EUR-Kurs: 0.8763


,Name,Set,Rarity,Price USD,Price EUR,Oracle Text,Image URL
0,Lightning Bolt,Ravnica: Clue Edition,uncommon,0.86,0.75,Lightning Bolt deals 3 damage to any target.,https://cards.scryfall.io/normal/front/7/7/77c...


✅ CSV gespeichert: Lightning_Bolt_info.csv
